In [1]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import DonutProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from warnings import filterwarnings
from transformers import VisionEncoderDecoderModel, AutoTokenizer
filterwarnings('ignore')
from tqdm import tqdm
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import torch
import pandas as pd
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel
from tqdm import tqdm
import requests
from io import BytesIO

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Path to your checkpoint directory
checkpoint_path = "/home/arjun/Desktop/Github/AmazonML-Hackathon/Donut_tests/donut-finetuned-docvqa/checkpoint-4500"

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
# Load the processor from the checkpoint
# processor = DonutProcessor.from_pretrained(checkpoint_path)

# Determine the device to use
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model from the checkpoint and move it to the appropriate device
# model = VisionEncoderDecoderModel.from_pretrained(checkpoint_path).to(device)

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa").to(device)

In [3]:
def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

def perform_inference(image, question):
    # Prepare the input
    image = image.convert('RGB')
    encoding = processor(images=image, text=question, return_tensors="pt")
    
    # Move input to the correct device
    for k, v in encoding.items():
        encoding[k] = v.to(device)
    # print('start giving outputs')
    # print(encoding['pixel_values'].shape)
    # Generate the output
    outputs = model.generate(
        pixel_values=encoding["pixel_values"],
        # decoder_input_ids=encoding["decoder_input_ids"],
        max_length=model.config.max_length,
        early_stopping=True,
        pad_token_id=model.config.pad_token_id,
        eos_token_id=model.config.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # outputs = model.generate(
    #     pixel_values.to(device),
    #     decoder_input_ids=decoder_input_ids.to(device),
    #     max_length=model.decoder.config.max_position_embeddings,
    #     pad_token_id=processor.tokenizer.pad_token_id,
    #     eos_token_id=processor.tokenizer.eos_token_id,
    #     use_cache=True,
    #     bad_words_ids=[[processor.tokenizer.unk_token_id]],
    #     return_dict_in_generate=True,)

    # Decode the output
    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = sequence.strip()

    return sequence

In [4]:
test_df = pd.read_csv('/home/arjun/Desktop/Github/AmazonML-Hackathon/dataset/test.csv')

# Define the entity_unit_map (as in your training script)
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}


In [5]:
def get_local_image_path(image_link, image_folder):
    # Extract filename from the URL
    filename = image_link.split('/')[-1]
    # Construct the full path to the local image file
    local_path = os.path.join(image_folder, filename)
    return local_path

image_folder = '/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test_images'


In [6]:
from print_color import print
import matplotlib.pyplot as plt

In [7]:
results = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = get_local_image_path(row['image_link'], image_folder)
    # print(image_path)
    
    if row['entity_name'] in entity_unit_map:
        potential_units = entity_unit_map[row['entity_name']]
        question = f"What is the {row['entity_name']} of the item in {potential_units}: "
    else:
        question = f"What is the {row['entity_name']}: "
    
    # print(f"Processing image: {image_path}")
    # # print(f"Question: {question}")
    image = Image.open(image_path)
    # plt.imshow(image)
    # plt.show()
    prediction = perform_inference(image, question)
    # print(prediction,color='red')
    results.append({'index': row['index'], 'prediction': prediction})

  1%|          | 1026/131187 [10:37<22:27:43,  1.61it/s]


KeyboardInterrupt: 

In [ ]:
results

In [ ]:
output_df = pd.DataFrame(results)

In [ ]:

# Save the results to a CSV file
output_df.to_csv('inference_results.csv', index=False)

print("Inference completed. Results saved to 'inference_results.csv'")

In [ ]:
# results = []
# for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
#     image_link = download_image(row['image_link'])
#     img_files = os.listdir('/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test_images')
#     if row['entity_name'] in entity_unit_map:
#         potential_units = entity_unit_map[row['entity_name']]
#         question = f"Given the image, what is the {row['entity_name']} of the item in {potential_units}: "
#     else:
#         question = f"Given the image, what is the {row['entity_name']}: "
#     print(image, question)
#     prediction = perform_inference(image, question)
#     results.append({'index': row['index'], 'prediction': prediction})


In [ ]:
results